In [1]:
%load_ext autoreload
%autoreload 2

In [107]:
import numpy as np
import pandas as pd
import os

import torch
from sklearn.linear_model import LinearRegression

import pdb

In [137]:
def estimate_expectation(data):
    estimate = torch.mean(data)
    return estimate

def compute_alpha(Z, X, max_m=6):
    deg_m = 2
    var_u = estimate_expectation(Z*X)
    sign = torch.sign(var_u)
    alpha2 = None
    alpha_estimations = []
    
    while deg_m<max_m:
        diff_normal_X = estimate_expectation(X**(deg_m)*Z) - deg_m*var_u*estimate_expectation(X**(deg_m-1))
        diff_normal_Z = estimate_expectation(Z**(deg_m)*X) - deg_m*var_u*estimate_expectation(Z**(deg_m-1))
        
        alpha_sq = ((diff_normal_X) / (diff_normal_Z))
        if alpha_sq < 0:
            alpha_sq = -(abs(alpha_sq)**(1/(deg_m-1)))
        else:
            alpha_sq = alpha_sq**(1/(deg_m-1))
        alpha_sq = abs(alpha_sq)*sign
        alpha_estimations.append((alpha_sq, deg_m))
        deg_m += 1
    
    return alpha_estimations

def compute_beta(Z, X, Y, alpha_sq):
    numerator = estimate_expectation(X*Y) - alpha_sq*estimate_expectation(Y*Z)
    denominator = estimate_expectation(X*X) - alpha_sq*estimate_expectation(X*Z)
    return numerator / denominator

In [138]:
file = "data/krueger/krueger_fast_food.csv"
file_path = os.path.join(os.getcwd(), file)
data = pd.read_csv(file_path)

data_pre_treat = data[data["Treatment"]==0].reset_index()
data_post_treat = data[data["Treatment"]==1].reset_index()
assert (data_pre_treat["Group"]==data_post_treat["Group"]).all(), "Discrepancy in data"

Z = torch.from_numpy(data_pre_treat["Empl"].to_numpy())
X = torch.from_numpy(data_pre_treat["Group"].to_numpy(dtype=np.float32))
Y = torch.from_numpy(data_post_treat["Empl"].to_numpy())

mask = torch.logical_or(Z.isnan(), X.isnan())
mask = torch.logical_or(mask,  Y.isnan())
mask = torch.logical_not(mask)
Z = Z[mask]
X = X[mask]
Y = Y[mask]


Z -= estimate_expectation(Z)
X -= estimate_expectation(X)
Y -= estimate_expectation(Y)

### Estimation of beta by our method

In [171]:
max_deg = 40
betas_est = []
alpha_estimations = compute_alpha(Z, X, max_deg)

In [173]:
for (alpha_sq, deg) in alpha_estimations:
    beta = compute_beta(Z, X, Y, alpha_sq)
    betas_est.append(beta)
    print(f"For degree {deg} we estimate beta as: {beta}")

For degree 2 we estimate beta as: 3.985089435582578
For degree 3 we estimate beta as: 2.127888108632464
For degree 4 we estimate beta as: 2.3806719938332073
For degree 5 we estimate beta as: 3.3704736599676437
For degree 6 we estimate beta as: 5.19234204452204
For degree 7 we estimate beta as: 3.8699104401743836
For degree 8 we estimate beta as: 3.4824169412230934
For degree 9 we estimate beta as: 3.3095408045253634
For degree 10 we estimate beta as: 3.203543927054919
For degree 11 we estimate beta as: 3.1291460963364623
For degree 12 we estimate beta as: 3.072767289179275
For degree 13 we estimate beta as: 3.0279301533696907
For degree 14 we estimate beta as: 2.9910583565726396
For degree 15 we estimate beta as: 2.9599847754816864
For degree 16 we estimate beta as: 2.933301984878247
For degree 17 we estimate beta as: 2.910047538939887
For degree 18 we estimate beta as: 2.889536231840652
For degree 19 we estimate beta as: 2.871264163606398
For degree 20 we estimate beta as: 2.854850901

In [174]:
# deg = max_deg-1
# print(f"For {deg} beta is estimated to be: {betas_est[-1]}")

### Estimation of beta with TFWE

In [58]:
data = data.dropna()
data["PostTreatment"] = data["Group"]*data["Treatment"]
data.head()


,ID,Chain,SouthJ,CentralJ,NorthJ,PA1,PA2,Shore,Group,Empl,C.Owned,Hours.Opening,Soda,Fries,Treatment,PostTreatment
0,46,1,0,0,0,1,0,0,0,30.0,0,16.5,1.03,1.03,0,0
1,49,2,0,0,0,1,0,0,0,6.5,0,13.0,1.01,0.90,0,0
2,506,2,0,0,0,1,0,0,0,3.0,1,10.0,0.95,0.74,0,0
3,56,4,0,0,0,1,0,0,0,20.0,1,12.0,0.87,0.82,0,0
4,61,4,0,0,0,1,0,0,0,6.0,1,12.0,0.87,0.77,0,0


In [79]:
X = data.drop(["ID", "Chain", "Empl"], axis=1).to_numpy()
Y = data["Empl"].to_numpy()

reg = LinearRegression().fit(X, Y)
reg.score(X, Y)
beta = reg.coef_[-1]
print("Beta estimate by linear regression: ", beta)

Beta estimate by linear regression:  4.140920858168537
